In [ ]:
from tqdm import tqdm
import torch
import torchvision
from torchvision import transforms, datasets

from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cl = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout(),

            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout(),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(6272, 10) # 10 means num_labels
        )

    def forward(self, xb):
        feat = self.cl(xb)
        return self.fc(feat)

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Training on', device)

Training on cpu


In [ ]:
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)


# split to train & val
train_data_size = int(len(train_data) * 0.9)
val_data_size = len(train_data) - train_data_size
train_data, val_data = data.random_split(train_data, [train_data_size, val_data_size])

# set batch size and load dataset into dataloader
batch_size = 256

print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Val Data : {len(val_data)}")

train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_data, batch_size*2, num_workers = 4, pin_memory = True)

100%|██████████| 26421880/26421880 [00:02<00:00, 12504610.41it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 214686.11it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3966289.75it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4576574.18it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Length of Train Data : 54000
Length of Val Data : 6000



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
model = CNN()
model = model.to(device)

# find best epoch, lr hyperparameter
num_epochs = 20
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr)

In [ ]:
# start training
for epoch in range(num_epochs):

    # training
    model.train()
    for images, labels in tqdm(train_dl):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        out = model(images)                  # Generate predictions
        train_loss = F.cross_entropy(out, labels) # Calculate loss
        train_acc = accuracy(out, labels)
        train_loss.backward()
        optimizer.step()

    # validation
    model.eval()
    for images, labels in tqdm(val_dl):
        images = images.to(device)
        labels = labels.to(device)
        out = model(images)                    # Generate predictions
        val_loss = F.cross_entropy(out, labels)   # Calculate loss
        val_acc = accuracy(out, labels)           # Calculate accuracy

    print('Epoch', epoch, '\nTrain Loss', f'\t{train_loss.item():.4f}', 'Train Acc', f'\t{train_acc.item():.4f}', '\nVal Loss  ', f'\t{val_loss.item():.4f}', 'Val Acc  ', f'\t{val_acc.item():.4f}')

100%|██████████| 12/12 [00:12<00:00,  1.08s/it]


Epoch 0 
Train Loss 	0.4054 Train Acc 	0.8333 
Val Loss   	0.3556 Val Acc   	0.8750


100%|██████████| 12/12 [00:13<00:00,  1.14s/it]


Epoch 1 
Train Loss 	0.4198 Train Acc 	0.8458 
Val Loss   	0.3327 Val Acc   	0.8859


100%|██████████| 12/12 [00:11<00:00,  1.01it/s]


...

Epoch 17 
Train Loss 	0.2534 Train Acc 	0.9042 
Val Loss   	0.1910 Val Acc   	0.9375


100%|██████████| 12/12 [00:12<00:00,  1.07s/it]


Epoch 18 
Train Loss 	0.2773 Train Acc 	0.8917 
Val Loss   	0.1898 Val Acc   	0.9212


100%|██████████| 12/12 [00:12<00:00,  1.03s/it]

Epoch 19 
Train Loss 	0.2386 Train Acc 	0.9000 
Val Loss   	0.2149 Val Acc   	0.9239
